In [6]:
# General purpose
import pandas as pd
import os
import glob
import json
import numpy as np
import string
import matplotlib.pyplot as plt
from datetime import datetime
from ast import literal_eval
#word2vec
from gensim.models.word2vec import Word2Vec
#sklearn related
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score

import embedding
import polarity_induction_methods as pi

# Hamilton - Lexicon 1

We have to define an embedding first

In [ ]:
#setting the correct working directory
# os.chdir("./1_ScoreInduction")

# Open the pre-prepared lexicon
with open("../lexicon/lexicon_refined.csv", 'r') as file:
        lexicon_refined = pd.read_csv(file, engine='c')

    # change column name
lexicon_refined = lexicon_refined.rename(columns={"Unnamed: 0": "Words"})

#loading models and selecting
slices = {
          filename.split('/')[-1].replace(".model", "") : 
          Word2Vec.load(filename)
          for filename in glob.glob('../models/*.model')
         }

## Testing

In [50]:
import embedding
import polarity_induction_methods as pi

In [51]:
###########################################
# DEFINING POSITIVE-NEGATIVE SEEDS        #
###########################################
positive_seed = list(enriched_lexicon[enriched_lexicon["Labels"] == 1]["Words"])[:50]
negative_seed = list(enriched_lexicon[enriched_lexicon["Labels"] == -1]["Words"])

In [19]:

#vecs is a list of vectors of the embeddings
vecs = models.wv.vectors
#vocab is a list of the words
vocab = list(models.wv.vocab.keys())

###########################################
# DEFINING EMBEDDING                      #
###########################################
NY_Embed = embedding.Embedding(vecs, vocab)

In [22]:
propagation = pi.random_walk(NY_Embed, positive_seed, negative_seed)

In [43]:
propagation_df = pd.DataFrame({"Words": [word 
                                         for word in propagation], 
                               "Label": [propagation[word] 
                                          for word in propagation]})

In [53]:
with open("../propagations/propagation_NYTimes_test.csv", 'w') as file:
    propagation_df.to_csv(file)

In [60]:
with open("../propagations/propagation_NYTimes_test.csv", 'r') as file:
    df = pd.read_csv(file, index_col=0)

## Propagare ALL

In [7]:
###########################################
# DEFINING POSITIVE-NEGATIVE SEEDS        #
###########################################
positive_seed = list(lexicon_refined[lexicon_refined["Valence"] == 1]["Words"])
negative_seed = list(lexicon_refined[lexicon_refined["Valence"] == -1]["Words"])

In [13]:
propagations = {sl: pi.random_walk(embedding.Embedding(slices[sl].wv.vectors, 
                                                       list(slices[sl].wv.vocab.keys())), 
                              positive_seed, 
                              negative_seed) 
                for sl in slices}

In [14]:
for name in slices:
    with open("../propagations/propagation_"+str(name)+".csv", 'w') as file:
        pd.DataFrame({
                     "Words": [word 
                                for word in propagations[name]],
                     "Labels": [propagations[name][word] 
                                for word in propagations[name]]
                     }).to_csv(file)

# Hamilton - Lexicon 2

In [2]:
#setting the correct working directory
# os.chdir("./1_ScoreInduction")

# Open the pre-prepared lexicon
with open("../lexicon_2/lexicon_refined.csv", 'r') as file:
        lexicon_refined = pd.read_csv(file, engine='c')

    # change column name
lexicon_refined = lexicon_refined.rename(columns={"Unnamed: 0": "Words"})

#loading models and selecting
slices = {
          filename.split('/')[-1].replace(".model", "") : 
          Word2Vec.load(filename)
          for filename in glob.glob('../models/*.model')
         }

In [3]:
###########################################
# DEFINING POSITIVE-NEGATIVE SEEDS        #
###########################################
positive_seed = list(lexicon_refined[lexicon_refined["Valence"] == 1]["Words"])
negative_seed = list(lexicon_refined[lexicon_refined["Valence"] == 0]["Words"])

In [4]:
propagations = {sl: pi.random_walk(embedding.Embedding(slices[sl].wv.vectors, 
                                                       list(slices[sl].wv.vocab.keys())), 
                              positive_seed, 
                              negative_seed) 
                for sl in slices}

In [5]:
for name in slices:
    with open("../propagations_2/propagation_"+str(name)+".csv", 'w') as file:
        pd.DataFrame({
                     "Words": [word 
                                for word in propagations[name]],
                     "Labels": [propagations[name][word] 
                                for word in propagations[name]]
                     }).to_csv(file)